In [3]:
import pyaudio
import wave
import os
from transformers import pipeline
import pywhisper
import torch

In [4]:
FRAMES=1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RUTA_GRABACIONES= "./grabaciones"
os.makedirs(RUTA_GRABACIONES, exist_ok=True)
p = pyaudio.PyAudio()

stream = p.open(
                format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=FRAMES
                )
print(f"Comienzo de la grabación...")
frames=[]
segundos=10
for j in range(0,int(RATE/FRAMES*segundos)):
    data=stream.read(FRAMES)
    frames.append(data)

print(f"Fin de la grabación...")
stream.stop_stream()
stream.close()
filename = os.path.join(RUTA_GRABACIONES, f"grabacion.wav")
with wave.open(filename, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(2)  
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

p.terminate()

print("Grabacion completadas.")

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

def transcribir_audio(archivo_audio):
    modelo = pywhisper.load_model("base", device=device)  # Puedes ajustar el modelo según tus necesidades
    resultado = modelo.transcribe(archivo_audio)
    return ' '.join(dic['text'] for dic in resultado['segments'])

In [6]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
def summarize_long_text(text, chunk_size=512):
    """Summarizes a long text by splitting it into chunks."""
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    summary_chunks = []

    for chunk in chunks:
        summary_chunks.append(summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])

    return " ".join(summary_chunks)

Device set to use cpu


In [7]:
transcripcion = transcribir_audio("./grabaciones/grabacion.wav")
print(transcripcion)

100%|███████████████████████████████████████| 139M/139M [00:05<00:00, 28.7MiB/s]
c:\Users\rafae\Desktop\AINE_AUDIO_SUMMARIZE\.venv\Lib\site-packages\pywhisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Hola, esto es una prueba, prueba probando, probando, lista de la compra, huevos, leche,  feriales y miel.


In [8]:
resumen = summarize_long_text(transcripcion)
print(resumen)

Your max_length is set to 130, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


Lista de la compra, huevos, leche,  feriales y miel. Hola, esto es una prueba.


### Validación

In [11]:
transcripcion = transcribir_audio("./grabaciones/WhatsApp Audio 2025-05-10 at 17.24.20_2448ebda.opus")
print(transcripcion)
resumen = summarize_long_text(transcripcion)
print(resumen)

Your max_length is set to 130, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


 Bajaro de los Nacionales volando por arriba de la feria de Sevilla
Bajaro de los Nacionales volando por arriba de la feria de Sevilla. Bajaro   volando  por ariba de the feria.


In [23]:
import av
import torch
import numpy as np
from transformers import LlavaNextVideoProcessor, LlavaNextVideoForConditionalGeneration

model_id = "llava-hf/LLaVA-NeXT-Video-7B-hf"

model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(device)

processor = LlavaNextVideoProcessor.from_pretrained(model_id)

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


# define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image", "video") 
conversation = [
    {

        "role": "user",
        "content": [
            {"type": "text", "text": "Describe el video."},
            {"type": "video"},
            ],
    },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

video_path = "./grabaciones/videoplayback.mp4"
container = av.open(video_path)

# sample uniformly 8 frames from the video, can sample more for longer videos
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip = read_video_pyav(container, indices)
inputs_video = processor(text=prompt, videos=clip, padding=True, return_tensors="pt").to(model.device)

output = model.generate(**inputs_video, max_new_tokens=100, do_sample=False)
print(processor.decode(output[0][2:], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 3/3 [00:49<00:00, 16.56s/it]


ER: 
Describe el video. ASSISTANT: The video shows a group of people dressed in religious attire, including robes and hats, sitting in rows and listening to a speaker who is likely a religious figure, possibly a priest or a bishop, given the context. The setting appears to be a formal religious event, possibly a sermon or a lecture, given the presence of the religious figure and the attire of the attendees. The attendees are focused on the speaker, and the speaker is addressing them


In [24]:
resumen = summarize_long_text(processor.decode(output[0][2:], skip_special_tokens=True))

Your max_length is set to 130, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


In [25]:
print(resumen)

Video shows a group of people dressed in religious attire. The setting appears to be a formal religious event. The attendees are focused on the speaker, and the speaker is addressing them.
